In [ ]:
%load_ext autoreload
%autoreload 2
import numpy as np # math
import matplotlib.pyplot as plt # graphs and plots
import pandas as pd
from tqdm import tqdm
import datetime
import matplotlib.dates as mdates
from src.utils.profiles_from_schedule_new import SimParam # class for show-up generator

In [ ]:
simparam = SimParam()
path_to_6k_sched = (
    r"/home/antoine/projects/"
    "KAPpaxsim/KAPpaxsim/data/"
    "processed/Schedule (30th terminal peak, 6000 pax)_PROCESSED.xlsx"
)
simparam.schedule_from_path(path_to_6k_sched)
simparam.schedule_cleanup()
simparam.schedule_filter()

simparam.show_up_from_file()
simparam.assign_flight_show_up_category_default()

In [ ]:
simparam.plot_show_up_categories_profiles()

In [ ]:
simparam.assign_show_up()

In [ ]:
simparam.plot_df_Pax()

In [ ]:
simparam.assign_check_in()

In [ ]:
import seaborn as sns
simparam.df_Counters['total'].plot()

In [ ]:
test = simparam.df_Counters.drop(columns='total')
sns.heatmap(test)

In [ ]:
simparam.dct_resource = {
    'kiosk' : 92
}

simparam.dct_processes = {
    'kiosk' : 5
}

In [ ]:
simparam.df_Pax

In [ ]:
simparam.prepare_schedule_for_simulation()

In [ ]:
from src.simfunc.KIX_T1d_CUSBD_new import Pax, Simulation

In [ ]:
simulation = Simulation(simparam)
simulation.generate_pax()

In [ ]:
end_time = 1500

with tqdm(total=end_time - 1, desc="Simulation running...") as runpbar:
    for i in range(1, end_time):
        simulation.env.run(until=i)
        runpbar.update(1)



In [ ]:
df = pd.concat([ pax.row for pax in simulation.pax_list],axis=1).transpose()


In [ ]:
simulation.format_df_result()

In [ ]:
simulation.plot_result()